### check
   * exchange balance of bitocin v.s. stables
   * large transfer in of stables v.s. bitcoins -> large trading volume? -> cause price move?

In [8]:
from cdcqr.data.glassnode import glassnode_data as gnd
from cdcqr.ct.utils import plot2
import pandas as pd
import cdcqr.analytics.utils as autil 
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from cdcqr.analytics.utils import calculate_corr_pvalues
from cdcqr.common.utils import LOCAL_DATA_DIR
import os
import numpy as np
from cdcqr.backtest.vbt.runsignal import runsignal
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import time
from IPython.display import Audio
sound_file = '/core/tmp/ding.wav'
%matplotlib inline


### Load data

In [4]:
gn = gnd.GlassnodeData()

In [5]:
gn.get_feature_best_resolutions('balance_exchanges')

'10m'

In [6]:
fs = ['price_usd_close', 'balance_exchanges']
assets = ['BTC','USDT', 'USDC', 'BUSD']
df = gn.load_features(fs, assets, resolution='10m')

loading market/price_usd_close asset=BTC, resolution=10m
loading market/price_usd_close asset=USDT, resolution=10m
loading market/price_usd_close asset=USDC, resolution=10m
loading market/price_usd_close asset=BUSD, resolution=10m
loading distribution/balance_exchanges asset=BTC, resolution=10m
loading distribution/balance_exchanges asset=USDT, resolution=10m
loading distribution/balance_exchanges asset=USDC, resolution=10m
loading distribution/balance_exchanges asset=BUSD, resolution=10m


In [7]:
df1 = df[df.index>='20200101'].ffill().drop(['USDT-price_usd_close','USDC-price_usd_close','BUSD-price_usd_close'],axis=1)
df1['stable-balance_exchanges'] = df['USDT-balance_exchanges']+df['USDC-balance_exchanges']+df['BUSD-balance_exchanges']
df1 = df1[['BTC-price_usd_close', 'stable-balance_exchanges','BTC-balance_exchanges']]
df1.head()

,BTC-price_usd_close,stable-balance_exchanges,BTC-balance_exchanges
t,,,
2020-01-01 00:00:00,7189.532563,8.618036e+08,3.027353e+06
2020-01-01 00:10:00,7182.082862,8.618286e+08,3.027422e+06
2020-01-01 00:20:00,7180.116033,8.617651e+08,3.027422e+06
2020-01-01 00:30:00,7180.654773,8.617208e+08,3.027469e+06
2020-01-01 00:40:00,7181.197965,8.618218e+08,3.027511e+06


In [8]:
df1['BTC-balance_exchanges_MC'] = df1['BTC-price_usd_close'] * df1['BTC-balance_exchanges']

In [9]:
df1['BTCSSR'] = df1['BTC-balance_exchanges_MC']/df1['stable-balance_exchanges']
df2 = df1.drop('BTC-balance_exchanges_MC', axis=1).dropna()

In [10]:
df3 = df2.copy()
df3['ret'] = df3['BTC-price_usd_close'].pct_change()
df3['r+5'] = df3['BTC-price_usd_close'].pct_change(periods=5).shift(-5)
df3['r+10'] = df3['BTC-price_usd_close'].pct_change(periods=10).shift(10)
df3['r+100'] = df3['BTC-price_usd_close'].pct_change(periods=100).shift(100)

df3['stable_bal_chg'] = df3['stable-balance_exchanges'].diff()
df3['stable_bal_chg_lag1'] = df3['stable_bal_chg'].shift(1)
df3['BTCSSR_lag1'] = df3['BTCSSR'].shift(1)
df3['BTCSSR_chg'] = df3['BTCSSR'].diff()
df3['BTCSSR_chg_lag1'] = df3['BTCSSR_chg'].shift(1)

In [11]:
df4 = df3.dropna()[['stable_bal_chg', 'stable_bal_chg_lag1', 'BTCSSR', 'BTCSSR_chg', 'BTCSSR_chg_lag1', 'ret', 'r+5', 'r+10', 'r+100']]

In [12]:
df4a = df4.where(~np.isinf(df4), 0)

In [13]:
lookback = 4000
t0 = time.time()
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
df4a['BTCSSR_pctrank4k'] = df4a['BTCSSR'].rolling(lookback).apply(pctrank)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

40.023335218429565


In [14]:
df4b = df4a[df4a.index>='20210101']

In [15]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))

### Load res df for analysis - long-BTCSSR_pctrank4k_res_df

In [ ]:
df = pd.read_pickle('/core/tmp/long-BTCSSR_pctrank4k_res_df.pickle')

In [ ]:
df.pv()

In [ ]:
feature = 'BTCSSR_pctrank4k'
p=price.resample('1Min').last().ffill()

In [ ]:
tp=15
sl=5
n1=None
n2=None
qtl=0.995
short=False
signal = df4b[feature]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
signal = signal.reindex(p.index).ffill()

ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])

In [ ]:
ressig['ma_tpsl_pf'].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

### Load res df for analysis - short-BTCSSR_pctrank4k_res_df

In [2]:
df = pd.read_pickle('/core/tmp/short-BTCSSR_pctrank4k_res_df.pickle')
df.pv()

In [19]:
t0 = time.time()
feature = 'BTCSSR_pctrank4k'
p=price.resample('1Min').last().ffill()
tp=7
sl=5
pf = 'rsi_tpsl_pf'
n1=None
n2=None
qtl=0.01
short=True
signal = df4b[feature]<qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
signal = signal.reindex(p.index).ffill()

ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,
                 rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])

print(time.time()-t0)

0.4937098026275635


In [22]:
ressig[pf].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#9467bd'},
              'na…

### load res_df files

In [17]:
res_df_fname1 = 'BTC_long-BTCSSR_pctrank_813d18d.pickle'
df1 = pd.read_pickle(os.path.join(LOCAL_DATA_DIR, 'backtest_res', res_df_fname))

In [21]:
df1

,signalid,short,tp,sl,k,lbw,qtl,sr,n1,n2,tr,ntrades
0,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.550695,1,101,-0.967435,54.688172
1,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.550695,1,1000,-0.967435,54.688172
2,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.550695,1,2000,-0.967435,54.688172
3,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.550695,1,500,-0.967435,54.688172
4,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.550695,1,200,-0.967435,54.688172
5,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.529927,10,500,-0.966856,54.580645
6,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.529927,10,101,-0.966856,54.580645
7,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.529927,10,200,-0.966856,54.580645
8,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.570695,100,200,-0.967975,54.645161
9,long-BTCSSR_pctrank,False,3,1,n_tpsl_pf,1000,0.995,-3.570695,100,2000,-0.967975,54.645161


In [22]:
df1.pv()